# Practice CNN with GPU support

- switch model, loss function and variables to GPU with `cuda()` method
- get data back from GPU with `cpu()` method, e.g. loss function

**In this MINIST case, the default CNN calculated with cpu takes ~300s, while ~50s in GPU mode.**

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
# 0 switch on/off gpu
use_gpu = torch.cuda.is_available()

In [3]:
# 1 create model
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential( # (1, 28, 28)
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2 ),      # (16, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),    # (16, 14, 14)
        )
        self.conv2 = nn.Sequential(  # (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),  # (32, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(2),  # (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # fully connected layer, output 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)   # (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output

net = CNN()
if use_gpu: net = net.cuda()

print(net)  # net architecture

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [11]:
# 2 load data
import torchvision
import torch.utils.data as Data

train_data = torchvision.datasets.MNIST(
    root='./dataset/', 
    train=True,
    transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST(
    root='./dataset/', 
    train=False)

train_data.data[0]

# batch train data
train_loader = Data.DataLoader(dataset=train_data, batch_size=32, shuffle=True)

# preprocess test data
# size: (n, 28, 28) -> (n, 1, 28, 28)
# value [0, 255] -> [0, 1]
with torch.no_grad():
    test_x = Variable(torch.unsqueeze(test_data.data, dim=1)).type(torch.FloatTensor) / 255.0
test_y = test_data.targets

if use_gpu:
    test_x = test_x.cuda()
    test_y = test_y.cuda()

In [12]:
# 3 train and evaluate model
fun_loss = nn.CrossEntropyLoss() # cross entropy loss
if use_gpu: 
    fun_loss = fun_loss.cuda()

optimizer = torch.optim.SGD(net.parameters(), lr=0.02) # SGD Optimizer


def evaluate(x, y):
    '''
    x: (n, 1, 28, 28)
    y: (n, 10)
    '''
    out = net(x)
    y_ = torch.max(out, 1)[1].detach() # max() return (value, index)
    accuracy = sum(y_==y) / y.size(0)
    return accuracy.item(), y_


# training and testing
for epoch in range(3):
    for step, (x, y) in enumerate(train_loader): 
        # batch x, y variables
        if use_gpu: 
            b_x = Variable(x.cuda())
            b_y = Variable(y.cuda())
        else:
            b_x = Variable(x)
            b_y = Variable(y)   

        output = net(b_x)               # ann output
        loss = fun_loss(output, b_y)    # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step%50 == 0:
            accuracy, _ = evaluate(test_x, test_y)
            if use_gpu: loss = loss.cpu()            
            print(f'epoch: {epoch} | loss: {loss.detach().item()} | accuracy: {accuracy}')

epoch: 0 | loss: 0.0757799744606018 | accuracy: 0.9825999736785889
epoch: 0 | loss: 0.04405032843351364 | accuracy: 0.9820999503135681
epoch: 0 | loss: 0.0028308623004704714 | accuracy: 0.9835999608039856
epoch: 0 | loss: 0.0035960767418146133 | accuracy: 0.9819999933242798
epoch: 0 | loss: 0.00780661404132843 | accuracy: 0.9829999804496765
epoch: 0 | loss: 0.022393101826310158 | accuracy: 0.9842000007629395
epoch: 0 | loss: 0.0037588062696158886 | accuracy: 0.982699990272522
epoch: 0 | loss: 0.041846923530101776 | accuracy: 0.98499995470047
epoch: 0 | loss: 0.1274210810661316 | accuracy: 0.9830999970436096
epoch: 0 | loss: 0.016850095242261887 | accuracy: 0.9819999933242798
epoch: 0 | loss: 0.08465032279491425 | accuracy: 0.9787999987602234
epoch: 0 | loss: 0.014006481505930424 | accuracy: 0.983199954032898
epoch: 0 | loss: 0.07772214710712433 | accuracy: 0.983199954032898
epoch: 0 | loss: 0.06476686149835587 | accuracy: 0.9797999858856201
epoch: 0 | loss: 0.004607530776411295 | accur